In [1]:
import numpy as np
import cv2 as cv
from ultralytics import YOLO

In [2]:
fourcc = cv.VideoWriter_fourcc(*'XVID')

cap = cv.VideoCapture('mud_dist.mkv')
res_L = cv.VideoWriter('res_L.avi', fourcc, 20.0, (640, 480))
res_R= cv.VideoWriter('res_R.avi', fourcc, 20.0, (640, 480))
saida= cv.VideoWriter('saida_L_R.avi', fourcc, 20.0, (640, 480))
while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frameL= frame[0:480,0:640]
    frameR =frame[0:480,640:1280]
    
    saida.write(frameL)
    saida.write(frameR)
    
    grayL = cv.cvtColor(frameL,cv.COLOR_BGR2GRAY)
    grayR=cv.cvtColor(frameR,cv.COLOR_BGR2GRAY)
    
    cv.imshow('frame R gray',grayR)
    cv.imshow('frame L gray',grayL)
    
    res_L.write(frameL)
    res_R.write(frameR)
    
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
res_L.release()
res_R.release()
saida.release()



Can't receive frame (stream end?). Exiting ...


In [7]:
# Load a model
model = YOLO('yolov8n.pt')  # load an official detection model

# Track with the model
#results = model.track(source="saida_L_R.avi", show=True, tracker="bytetrack.yaml") 
results = model.track(source="saida_L_R.avi", show=True)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/266) /home/sleepkat/saida_L_R.avi: 480x640 1 refrigerator, 49.1ms
video 1/1 (2/266) /home/sleepkat/saida_L_R.avi: 480x640 1 refrigerator, 47.0ms
video 1/1 (3/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 47.2ms
video 1/1 (4/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 47.1ms
video 1/1 (5/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.8ms
video 1/1 (6/266) /home/sleepkat/saida_L_R.avi: 480x640 (no d

video 1/1 (97/266) /home/sleepkat/saida_L_R.avi: 480x640 1 cup, 46.7ms
video 1/1 (98/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 47.0ms
video 1/1 (99/266) /home/sleepkat/saida_L_R.avi: 480x640 1 cup, 1 vase, 47.0ms
video 1/1 (100/266) /home/sleepkat/saida_L_R.avi: 480x640 1 cup, 46.8ms
video 1/1 (101/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.8ms
video 1/1 (102/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.8ms
video 1/1 (103/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.9ms
video 1/1 (104/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.7ms
video 1/1 (105/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.6ms
video 1/1 (106/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.8ms
video 1/1 (107/266) /home/sleepkat/saida_L_R.avi: 480x640 1 tv, 46.8ms
video 1/1 (108/266) /home/sleepkat/saida_L_R.avi: 480x640 (no detections), 46.8ms
video 1/1 (109/266) /home/sleepkat/saida_L_R.avi: 48

KeyboardInterrupt: 

In [3]:
cap = cv.VideoCapture('saida_L_R.avi')
model = YOLO('yolov8n.pt')  # load an official detection model
box_a = 0
xL = [0] * 20
xR = [0] * 20
old_xL = [0] * 20
old_xR = [0] * 20
salva = []
i = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.track(frame, persist=True, show=True)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    
    print(f"teste: {old_xR[0]}")
    
    for box, id in zip(boxes, ids):
        
        print(f"teste1: {old_xR[0]}")
        old_xL[id-1] = xL[id-1]
        old_xR[id-1] = xR[id-1]
        xL[id-1] = box[0]
        xR[id-1] = box[1]
      
        print(f"teste2: {old_xR[0]}")
        
        d_pixel = abs(old_xL[id-1] - xL[id-1])
        
        if d_pixel == 0:
            dist = 0
        else:
            dist = 13903.92/d_pixel
            
        cv.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv.putText(
            frame,
            f"Id:{id}",
            (box[0], box[1]),
            cv.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            )
        cv.putText(
            frame,
            f"D:{dist:.1f}cm",
            (box[0]+70, box[1]),
            cv.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            )
        cv.imshow("frame", frame)
        if cv.waitKey(1) & 0xFF == ord("q"):
            break


0: 480x640 1 refrigerator, 48.1ms
Speed: 20.4ms preprocess, 48.1ms inference, 280.1ms postprocess per image at shape (1, 3, 480, 640)


AttributeError: 'NoneType' object has no attribute 'cpu'